In [4]:
import importlib
import copy
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
# ----------------------------------------
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt import gp_minimize
# ----------------------------------------

import warnings
warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()


In [2]:
dataset='CTSlices'
CT_dat = np.genfromtxt("data/CTSlices/slice_localization_data.csv",
                       delimiter=",",
                       skip_header=1)
kf_num=10
X_train = CT_dat[:,1:-1]
y_train = CT_dat[:,-1]


In [3]:
def Batch_Update_CV(model, cv, X, Y):
    score_loc = np.zeros((cv.get_n_splits(),))
    for i, (train_index, test_index) in enumerate(cv.split(X)):
        model_loc = copy.deepcopy(model)
        try:
            for batch in gen_batches(len(train_index), len(test_index)):
                train_batch = train_index[batch]
                model_loc.fit(X[train_batch], Y[train_batch])

            y_true = Y[test_index]
            y_pred = model_loc.predict(X[test_index])
            score_loc[i] = RMSE(y_true, y_pred)
            # print(f"fold: {i:02d}, score:{score_loc[i]: .3f}")

        except Exception as e:
            print("-"*20)
            print(f"fold: {i:02d} failure !")
            print(" Error: ", e)
            score_loc[i] = np.nan
            break
    return np.mean(score_loc)

def Model_Score(model, train_index, test_index, X, Y):
    model.fit(X[train_index], Y[train_index])

    y_true = Y[test_index]
    y_pred = model.predict(X[test_index])

    return RMSE(y_true, y_pred)

In [4]:
from sklearn.cross_decomposition import PLSRegression

n_comp_lst=list(range(1,11))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLSRegression(scale=False), parameters,
                   cv=KFold(n_splits=kf_num),
                   scoring='neg_root_mean_squared_error')

print(clf)
tim_tot.tic()
clf.fit(X_train, y_train)
print(f"best parameter: {clf.best_params_}",
      f"; score: {-1.*clf.best_score_:15.7e}",
      f"; total time={tim_tot.tocvalue():.1f}s")

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=PLSRegression(scale=False),
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='neg_root_mean_squared_error')


best parameter: {'n_components': 9} ; score:   9.8758666e+00 ; total time=87.3s


In [19]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,11))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(), parameters, # verbose=3,
                   cv=KFold(n_splits=kf_num),
                   scoring='neg_root_mean_squared_error')

print(clf)
tim_tot.tic()
clf.fit(X_train, y_train)
print(f"best parameter: {clf.best_params_}",
      f"; score: {-1.*clf.best_score_:15.7e}",
      f"; total time={tim_tot.tocvalue():.1f}s")


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SIMPLS(),
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='neg_root_mean_squared_error')


best parameter: {'n_components': 9} ; score:   9.8758666e+00 ; total time=29.8s


In [20]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS

scores = []
params = []
n_comp_lst=list(range(1,11))
tim_tot.tic()
for n_comp in n_comp_lst:
    tim.tic()
    scores.append(Batch_Update_CV(ISIMPLS(n_components=n_comp),
                                  KFold(n_splits=kf_num),
                                  X_train, y_train))
    params.append({'n_comp': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

params={'n_comp': 1}, score=1.7601132e+01, elapsed time=2.7s


params={'n_comp': 2}, score=1.3769302e+01, elapsed time=2.7s


params={'n_comp': 3}, score=1.1244679e+01, elapsed time=2.7s


params={'n_comp': 4}, score=1.0709137e+01, elapsed time=2.7s


params={'n_comp': 5}, score=1.0220055e+01, elapsed time=2.8s


params={'n_comp': 6}, score=9.9753077e+00, elapsed time=2.8s


params={'n_comp': 7}, score=9.9088491e+00, elapsed time=2.8s


params={'n_comp': 8}, score=9.8766549e+00, elapsed time=2.8s


params={'n_comp': 9}, score=9.8758666e+00, elapsed time=2.8s


params={'n_comp': 10}, score=9.8766684e+00, elapsed time=2.9s

best parameter: {'n_comp': 9}; score: 9.8758666e+00; total time=28.0s


In [14]:
import Code.PLS1
importlib.reload(Code.PLS1)
from Code.PLS1 import PLS1

n_comp_lst=list(range(1,11))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLS1(), parameters, # verbose=3,
                   cv=KFold(n_splits=kf_num),
                   scoring='neg_root_mean_squared_error')

print(clf)
tim_tot.tic()
clf.fit(X_train, y_train)
print(f"best parameter: {clf.best_params_}",
      f"; score: {-1.*clf.best_score_:15.7e}",
      f"; total time={tim_tot.tocvalue():.1f}s")


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=PLS1(),
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='neg_root_mean_squared_error')


best parameter: {'n_components': 9} ; score:   9.8758666e+00 ; total time=33.8s


In [9]:
import Code.PLS1
importlib.reload(Code.PLS1)
from Code.PLS1 import IPLS1

scores = []
params = []
n_comp_lst=list(range(1,11))
tim_tot.tic()
for n_comp in n_comp_lst:
    tim.tic()
    scores.append(Batch_Update_CV(IPLS1(n_components=n_comp),
                                  KFold(n_splits=kf_num),
                                  X_train, y_train))
    params.append({'n_comp': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

params={'n_comp': 1}, score=1.7601132e+01, elapsed time=3.0s


params={'n_comp': 2}, score=1.3769302e+01, elapsed time=2.9s


params={'n_comp': 3}, score=1.1244679e+01, elapsed time=2.9s


params={'n_comp': 4}, score=1.0709137e+01, elapsed time=2.9s


params={'n_comp': 5}, score=1.0220055e+01, elapsed time=2.9s


params={'n_comp': 6}, score=9.9753077e+00, elapsed time=2.9s


params={'n_comp': 7}, score=9.9088491e+00, elapsed time=2.9s


params={'n_comp': 8}, score=9.8766549e+00, elapsed time=2.9s


params={'n_comp': 9}, score=9.8758666e+00, elapsed time=3.0s


params={'n_comp': 10}, score=9.8766684e+00, elapsed time=3.3s

best parameter: {'n_comp': 9}; score: 9.8758666e+00; total time=29.7s


In [15]:
import Code.CIPLS
importlib.reload(Code.CIPLS)
from Code.CIPLS import CIPLS

scores = []
params = []
n_comp_lst=list(range(1,11))
tim_tot.tic()
for n_comp in n_comp_lst:
    tim.tic()
    scores.append(Batch_Update_CV(CIPLS(n_components=n_comp),
                                  KFold(n_splits=kf_num),
                                  X_train, y_train))
    params.append({'n_comp': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

params={'n_comp': 1}, score=1.8478452e+01, elapsed time=197.7s


params={'n_comp': 2}, score=2.8084951e+01, elapsed time=392.8s


params={'n_comp': 3}, score=2.7448990e+01, elapsed time=587.3s


params={'n_comp': 4}, score=2.7028211e+01, elapsed time=786.5s


--------------------
fold: 01 failure !
 Error:  illegal value in 4th argument of internal gesdd
params={'n_comp': 5}, score=nan, elapsed time=195.9s


--------------------
fold: 00 failure !
 Error:  Input contains infinity or a value too large for dtype('float64').
params={'n_comp': 6}, score=nan, elapsed time=1.6s
--------------------
fold: 00 failure !
 Error:  Input contains infinity or a value too large for dtype('float64').
params={'n_comp': 7}, score=nan, elapsed time=0.0s
--------------------
fold: 00 failure !
 Error:  Input contains infinity or a value too large for dtype('float64').
params={'n_comp': 8}, score=nan, elapsed time=0.0s
--------------------
fold: 00 failure !
 Error:  Input contains infinity or a value too large for dtype('float64').
params={'n_comp': 9}, score=nan, elapsed time=0.0s
--------------------
fold: 00 failure !
 Error:  Input contains infinity or a value too large for dtype('float64').
params={'n_comp': 10}, score=nan, elapsed time=0.0s

best parameter: {'n_comp': 1}; score: 1.8478452e+01; total time=2162.0s


In [16]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

n_comp_max = 10

space  = [ Integer(1, 10, name='n_components'),
           Real(1e-9, 1e-5, name='mu', prior="log-uniform"),
           Real(0, 1, name='amnesic')]

@use_named_args(space)
def Comp_Model_Score(n_components, mu, amnesic):
    cv = KFold(n_splits=kf_num)
    score_fold = np.zeros((cv.get_n_splits(),))
    PLS = OLPLS(n_components=n_components, mu=mu, amnesic=amnesic)
    print(f"__n-comp={PLS.n_components:03d}, mu={PLS.mu:.4e}, amnesic={PLS.amnesic:.2f}")

    for i, (train_index, test_index) in enumerate(cv.split(X_train)):
        model_loc = copy.deepcopy(PLS)
        try:
            for batch in gen_batches(len(train_index), len(test_index)):
                train_batch = train_index[batch]
                model_loc.fit(X_train[train_batch], y_train[train_batch])

            y_true = y_train[test_index]
            y_pred = model_loc.predict(X_train[test_index], n_components)
            score_fold[i] = RMSE(y_true, y_pred)
            # print(f"fold: {i:02d}, score:{score_loc[i]: .3f}")

        except Exception as e:
            print("  "+"-"*20)
            print(f"  fold: {i:02d} failure !")
            print("    Error: ", e)
            print("  "+"-"*20)
            score_fold[i] = np.nan
            break
    return np.mean(score_fold)

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=30,
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best amnesic = {res_gp.x[0]}")
print(f"Best mu = {res_gp.x[1]}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")

Iteration No: 1 started. Evaluating function at random point.
__n-comp=006, mu=2.3827e-06, amnesic=0.86


  --------------------
  fold: 00 failure !
    Error:  illegal value in 4th argument of internal gesdd
  --------------------
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 61.0412
Function value obtained: nan
Current minimum: nan
Iteration No: 2 started. Evaluating function at random point.
__n-comp=009, mu=3.1207e-07, amnesic=0.38


  --------------------
  fold: 01 failure !
    Error:  illegal value in 4th argument of internal gesdd
  --------------------
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 185.0055
Function value obtained: nan
Current minimum: nan
Iteration No: 3 started. Evaluating function at random point.
__n-comp=004, mu=1.6860e-09, amnesic=0.27


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 400.7785
Function value obtained: 47.9161
Current minimum: nan
Iteration No: 4 started. Evaluating function at random point.
__n-comp=005, mu=1.7729e-06, amnesic=0.48


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 502.1539
Function value obtained: 46.5592
Current minimum: nan
Iteration No: 5 started. Evaluating function at random point.
__n-comp=005, mu=2.2096e-06, amnesic=0.34


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 715.4373
Function value obtained: 47.8602
Current minimum: nan
Iteration No: 7 started. Evaluating function at random point.
__n-comp=002, mu=3.0224e-06, amnesic=0.47


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 203.0047
Function value obtained: 42.7734
Current minimum: nan
Iteration No: 8 started. Evaluating function at random point.
__n-comp=008, mu=1.2076e-07, amnesic=0.68


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 826.6147
Function value obtained: 47.8434
Current minimum: nan
Iteration No: 9 started. Evaluating function at random point.
__n-comp=007, mu=2.1285e-07, amnesic=0.54


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 716.9673
Function value obtained: 47.6188
Current minimum: nan
Iteration No: 10 started. Evaluating function at random point.
__n-comp=008, mu=2.6523e-09, amnesic=0.47


ValueError: Input y contains NaN.

In [17]:
import Code.SGDPLS
importlib.reload(Code.SGDPLS)
from Code.SGDPLS import SGDPLS

space  = [ Integer(1, 10, name='n_components'),
           Real(1e-9, 1e-5, name='eta', prior="log-uniform") ]

@use_named_args(space)
def Comp_Model_Score(n_components, eta):
    cv = KFold(n_splits=kf_num)
    score_fold = np.zeros((cv.get_n_splits(),))
    PLS = SGDPLS(n_components=n_components, eta=eta)
    print(f"__n-comp={PLS.n_components:03d}, eta={PLS.eta:.4e}")

    for i, (train_index, test_index) in enumerate(cv.split(X_train)):
        model_loc = copy.deepcopy(PLS)
        try:
            for batch in gen_batches(len(train_index), len(test_index)):
                train_batch = train_index[batch]
                model_loc.fit(X_train[train_batch], y_train[train_batch])

            y_true = y_train[test_index]
            y_pred = model_loc.predict(X_train[test_index])
            score_fold[i] = RMSE(y_true, y_pred)
            # print(f"fold: {i:02d}, score:{score_loc[i]: .3f}")

        except Exception as e:
            print("  "+"-"*20)
            print(f"  fold: {i:02d} failure !")
            print("    Error: ", e)
            print("  "+"-"*20)
            score_fold[i] = np.nan
            break
    return np.mean(score_fold)

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=20,
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best amnesic = {res_gp.x[0]}")
print(f"Best mu = {res_gp.x[1]}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")

Iteration No: 1 started. Evaluating function at random point.
__n-comp=006, eta=2.3827e-06


  --------------------
  fold: 00 failure !
    Error:  Input contains NaN.
  --------------------
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 101.1086
Function value obtained: nan
Current minimum: nan
Iteration No: 2 started. Evaluating function at random point.
__n-comp=009, eta=2.4491e-06


  --------------------
  fold: 00 failure !
    Error:  Input contains NaN.
  --------------------
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 121.0916
Function value obtained: nan
Current minimum: nan
Iteration No: 3 started. Evaluating function at random point.
__n-comp=007, eta=3.4477e-08


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3472.5411
Function value obtained: 24062435.6205
Current minimum: nan
Iteration No: 4 started. Evaluating function at random point.
__n-comp=004, eta=1.6860e-09


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2780.4997
Function value obtained: 18152941.6886
Current minimum: nan
Iteration No: 5 started. Evaluating function at random point.
__n-comp=003, eta=8.1407e-08


Iteration No: 5 ended. Evaluation done at random point.
Time taken: 2549.2179
Function value obtained: 5726047.2883
Current minimum: nan
Iteration No: 6 started. Evaluating function at random point.
__n-comp=008, eta=8.3159e-08


Iteration No: 6 ended. Evaluation done at random point.
Time taken: 3697.9358
Function value obtained: 12288698.8261
Current minimum: nan
Iteration No: 7 started. Evaluating function at random point.
__n-comp=005, eta=2.2096e-06


  --------------------
  fold: 00 failure !
    Error:  Input contains NaN.
  --------------------
Iteration No: 7 ended. Evaluation done at random point.
Time taken: 128.4077
Function value obtained: nan
Current minimum: nan
Iteration No: 8 started. Evaluating function at random point.
__n-comp=004, eta=3.9146e-07


KeyboardInterrupt: 

In [ ]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

scores = []
params = []

mu_list = [1e-5, 1e-6, 1e-7, 1e-8]
amnesic_list = [1e-2] + list(np.arange(0.1,1.0,0.1)) + [0.99]

tim_tot.tic()
for mu in mu_list:
    for amnesic in amnesic_list:
        tim.tic()
        scores.append(Batch_Update_CV(OLPLS(n_components=9, mu=mu, amnesic=amnesic),
                                      KFold(n_splits=kf_num),
                                      X_train, y_train))
        params.append({'mu': mu, 'amnesic': amnesic})
        print(f"params={params[-1]}, score={scores[-1]:.7e}, "
              +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

 << Error >>  !!!!!!!!!!!!!!!!!!!!
illegal value in 4th argument of internal gesdd
fold: 00 failure !
params={'mu': 1e-05, 'amnesic': 0.01}, score=nan, elapsed time=95.9s


 << Error >>  !!!!!!!!!!!!!!!!!!!!
illegal value in 4th argument of internal gesdd
fold: 00 failure !
params={'mu': 1e-05, 'amnesic': 0.1}, score=nan, elapsed time=97.8s


 << Error >>  !!!!!!!!!!!!!!!!!!!!
illegal value in 4th argument of internal gesdd
fold: 00 failure !
params={'mu': 1e-05, 'amnesic': 0.2}, score=nan, elapsed time=98.0s


 << Error >>  !!!!!!!!!!!!!!!!!!!!
illegal value in 4th argument of internal gesdd
fold: 00 failure !
params={'mu': 1e-05, 'amnesic': 0.30000000000000004}, score=nan, elapsed time=97.7s


 << Error >>  !!!!!!!!!!!!!!!!!!!!
illegal value in 4th argument of internal gesdd
fold: 00 failure !
params={'mu': 1e-05, 'amnesic': 0.4}, score=nan, elapsed time=97.8s


...

params={'mu': 1e-08, 'amnesic': 0.6}, score=4.7902376e+01, elapsed time=985.0s


params={'mu': 1e-08, 'amnesic': 0.7000000000000001}, score=4.7902530e+01, elapsed time=984.6s


params={'mu': 1e-08, 'amnesic': 0.8}, score=4.7902711e+01, elapsed time=983.9s


params={'mu': 1e-08, 'amnesic': 0.9}, score=4.7902993e+01, elapsed time=985.9s


params={'mu': 1e-08, 'amnesic': 0.99}, score=4.7904304e+01, elapsed time=962.8s

best parameter: {'mu': 1e-05, 'amnesic': 0.99}; score: 4.0882449e+01; total time=26191.9s
